In [13]:
#sigmoid
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa
import random

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    #print(np.sum(S))
    
    if np.sum(S) >= 1000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=sample_rate, #取樣率
                                            winlen=0.025, #音框涵蓋時間(s)
                                            winstep=0.01, #音框間的時間(s)
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=512, #FFT大小
                                            lowfreq=0, #lowest band edge of mel filters. 
                                            highfreq=None, #highest band edge of mel filters. 
                                            preemph=0.97, #不用預強化濾波器
                                            ceplifter=22, #robust
                                            appendEnergy=True, #True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning) #漢明窗，增加音框左右端連續性
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        val = output_data[0][0]
        
        if val >= 0.8:
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str("贅詞") + "，" + str(val))
            data_yes.append(str(m) + "分" + str(s) + "秒，出現:" + str("贅詞") + "，" + str(val))
        else:
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str("非贅詞") + "，" + str(val))
#main
# Parameters
num_mfcc = 13 #回傳mfcc的量
model_path = './tflite/recordingSpecAugmentSigmoid.tflite'

data = []
data_yes = []
start = 0 #一開始的索引值
end = 4000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 16000 #取樣率

#載入音檔
y, sr = librosa.load("./TestWAV/chen_16000HZ.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

while True:
    s = s + 0.25
    if(s >= 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    window[:4000] = window[4000:] #把音訊載入window
    window[4000:] = y[start:end] #把音訊載入window
    
    sound(window,s,m) #呼叫sound()

    if(end == (16000 * duration)): #如果移動到最後，break
        break
    
    start = start + 4000 #向後移動
    end = end + 4000 #向後移動   
    
    
for i in range(len(data)):
        print(data[i])

print(len(data))
print(str(len(data) - len(data_yes)))
    

0分1.5秒，出現:非贅詞，0.59116006
0分1.75秒，出現:非贅詞，0.646173
0分2.0秒，出現:贅詞，0.9633721
0分2.25秒，出現:贅詞，0.8957192
0分2.5秒，出現:贅詞，0.99374115
0分2.75秒，出現:贅詞，0.9802555
0分3.0秒，出現:贅詞，0.99990106
0分3.25秒，出現:非贅詞，0.7123499
0分3.5秒，出現:非贅詞，0.087856896
0分3.75秒，出現:贅詞，0.9069205
0分4.0秒，出現:非贅詞，0.56983376
0分4.25秒，出現:非贅詞，0.7118702
0分4.5秒，出現:非贅詞，0.6979728
0分4.75秒，出現:非贅詞，0.40188038
0分5.0秒，出現:非贅詞，0.615834
0分5.25秒，出現:贅詞，0.84789145
0分5.5秒，出現:贅詞，0.80510885
0分5.75秒，出現:贅詞，0.8591263
0分6.0秒，出現:非贅詞，0.5045567
0分6.25秒，出現:贅詞，0.930567
0分6.5秒，出現:非贅詞，0.77124345
0分6.75秒，出現:非贅詞，0.25111246
0分7.0秒，出現:非贅詞，0.5854246
0分7.25秒，出現:非贅詞，0.5179652
0分7.5秒，出現:贅詞，0.9528415
0分7.75秒，出現:非贅詞，0.6701684
0分8.0秒，出現:贅詞，0.8112136
0分8.25秒，出現:贅詞，0.8572783
0分8.5秒，出現:贅詞，0.8998077
0分8.75秒，出現:非贅詞，0.69128036
0分9.0秒，出現:非贅詞，0.5509076
0分9.25秒，出現:非贅詞，0.5216026
0分9.5秒，出現:贅詞，0.9999166
0分9.75秒，出現:贅詞，0.9983503
0分10.0秒，出現:非贅詞，0.00073083484
0分10.25秒，出現:贅詞，0.9534161
0分10.5秒，出現:非贅詞，0.29007727
0分10.75秒，出現:非贅詞，0.41618282
0分11.0秒，出現:非贅詞，0.7955791
0分11.25秒，出現:贅詞，0.8914683
0分11.5秒，出現:贅詞，0.8